In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
# Import the file

FILE_PATH = "C:/Users/nann4/OneDrive/Desktop/Study/Survival Analysis/Dashboard/Dataset/BrCa Cases 2012-16 18 SEER Areas.txt"
df = pd.read_csv(FILE_PATH, sep="	")
df.head()

C:\Users\nann4\AppData\Local\Temp\ipykernel_32776\3745682814.py:4: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(FILE_PATH, sep="	")


,Patient ID,Year_of_diagnosis,Age_of_diagnosis,Race,Origin,Histologic_Type,Combined_Summary_Stage_2000,Grade,Survival_months,Vital_status_recode
0,321,2015,54,White,Non-Spanish-Hispanic-Latino,8522,Regional lymph nodes involved only,Moderately differentiated; Grade II,0012,Alive
1,357,2013,44,White,Non-Spanish-Hispanic-Latino,8010,Unknown/unstaged/unspecified/DCO,Unknown,0028,Dead
2,390,2012,82,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,0051,Alive
3,405,2014,56,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,0033,Alive
4,581,2013,71,White,Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,0038,Alive


In [17]:
# Rename the Patient ID into Patient_ID

df = df.rename(columns={"Patient ID": "Patient_ID"})
df = df.rename(columns={"Year_of_diagnosis": "YearDx"})
df = df.rename(columns={"Age_of_diagnosis": "AgeDx"})
df = df.rename(columns={"Combined_Summary_Stage_2000": "Stage"})



In [18]:
# Change the columns into numeric

df["AgeDx"] = pd.to_numeric(df["AgeDx"])
df["Survival_months"] = pd.to_numeric(df["Survival_months"], errors="coerce")
df["YearDx"] = pd.to_numeric(df["YearDx"])

In [19]:
df.head()

,Patient_ID,YearDx,AgeDx,Race,Origin,Histologic_Type,Stage,Grade,Survival_months,Vital_status_recode
0,321,2015,54,White,Non-Spanish-Hispanic-Latino,8522,Regional lymph nodes involved only,Moderately differentiated; Grade II,12.0,Alive
1,357,2013,44,White,Non-Spanish-Hispanic-Latino,8010,Unknown/unstaged/unspecified/DCO,Unknown,28.0,Dead
2,390,2012,82,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,51.0,Alive
3,405,2014,56,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,33.0,Alive
4,581,2013,71,White,Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,38.0,Alive


In [20]:
df[['AgeDx','Survival_months']].describe()

,AgeDx,Survival_months
count,319823.000000,319790.000000
mean,61.927597,25.862904
std,13.554400,17.209613
min,8.000000,0.000000
25%,52.000000,11.000000
50%,62.000000,24.000000
75%,71.000000,40.000000
max,118.000000,59.000000


In [21]:
categorical_cols = ['Race', 'Origin', 'Grade', 'Vital_status_recode']

for col in categorical_cols:
    print(f"\n🧠 {col}:")
    print(df[col].value_counts(dropna=False))


🧠 Race:
Race
White                            250797
Black                             36083
Asian or Pacific Islander         28055
Unknown                            2972
American Indian/Alaska Native      1916
Name: count, dtype: int64

🧠 Origin:
Origin
Non-Spanish-Hispanic-Latino    282971
Spanish-Hispanic-Latino         36852
Name: count, dtype: int64

🧠 Grade:
Grade
Moderately differentiated; Grade II       134978
Poorly differentiated; Grade III           94345
Well differentiated; Grade I               69850
Unknown                                    19819
Undifferentiated; anaplastic; Grade IV       831
Name: count, dtype: int64

🧠 Vital_status_recode:
Vital_status_recode
Alive    290674
Dead      29149
Name: count, dtype: int64


In [22]:
print(df["Grade"].unique())

['Moderately differentiated; Grade II' 'Unknown'
 'Poorly differentiated; Grade III' 'Well differentiated; Grade I'
 'Undifferentiated; anaplastic; Grade IV']


In [23]:
print(df['Stage'].unique())

['Regional lymph nodes involved only' 'Unknown/unstaged/unspecified/DCO'
 'Localized only' 'Distant site(s)/node(s) involved'
 'Regional by direct extension only'
 'Regional by both direct extension and lymph node involvement'
 'Regional, NOS']


In [24]:
print(df['Histologic_Type'].unique())

[8522 8010 8500 8510 8480 8520 8523 8046 8211 8530 8140 8050 8524 8000
 9020 8401 8260 8575 8200 9120 8503 8255 8507 8521 8540 8201 8541 8504
 9180 8501 8032 8543 8574 8246 8005 8070 8560 8890 8980 8323 8453 8343
 8481 8514 8800 8570 8502 8230 8344 8550 8243 8830 8490 8982 8249 8035
 8720 8935 8513 8801 8810 8802 8022 8525 8571 8041 8033 8310 8572 8240
 8071 8805 8045 8082 8512 8983 8013 8074 8315 8804 8141 8573 8811 8430
 8542 8021 8001 8470 8004 8072 8012 8020 8410 9040 8123 8440 8815 8851
 8110 8832 9041 9580 8900 8854 8130 9220 8852 8251 8290 8471 8147 8052
 8562 8850 9170 8030 9183 8460 8341 8990 8940 8920]


In [25]:
# Mapping: histology_code -> (histology_name, histology_category)

histology_mapping = {
    # Neoplasm
    8000: ("Neoplasm, malignant", "Neoplasm"),
    8001: ("Tumor cells, malignant", "Neoplasm"),
    8002: ("Malignant tumor, small cell type", "Neoplasm"),
    8003: ("Malignant tumor, giant cell type", "Neoplasm"),
    8004: ("Malignant tumor, spindle cell type", "Neoplasm"),
    8005: ("Malignant tumor, clear cell type", "Neoplasm"),
    # Carcinoma, NOS
    8010: ("Carcinoma, NOS", "Carcinoma"),
    8011: ("Epithelioma, malignant", "Carcinoma"),
    8012: ("Large cell carcinoma, NOS", "Carcinoma"),
    8013: ("Large cell neuroendocrine carcinoma", "Carcinoma"),
    8014: ("Large cell carcinoma with rhabdoid phenotype", "Carcinoma"),
    8015: ("Glassy cell carcinoma", "Carcinoma"),
    # Carcinoma, undiff., NOS
    8020: ("Carcinoma, undifferentiated type, NOS", "Carcinoma"),
    8021: ("Carcinoma, anaplastic type, NOS", "Carcinoma"),
    8022: ("Pleomorphic carcinoma", "Carcinoma"),
    8023: ("Carcinoma, anaplastic type, NOS", "Carcinoma"),
    # Giant & Spindle Cell Carcinoma
    8030: ("Giant cell carcinoma", "Carcinoma"),
    8031: ("Spindle cell carcinoma", "Carcinoma"),
    8032: ("Pseudosarcomatous carcinoma", "Carcinoma"),
    8033: ("Polygonal cell carcinoma", "Carcinoma"),
    8034: ("Carcinoma with osteoclast-like giant cells", "Carcinoma"),
    8035: ("Carcinoma with osteoclast-like giant cells", "Carcinoma"),
    # Small Cell Carcinoma, NOS
    8041: ("Small cell carcinoma, NOS", "Carcinoma"),
    8043: ("Small cell carcinoma, fusiform cell", "Carcinoma"),
    8045: ("Combined small cell carcinoma", "Carcinoma"),
    # Epithelial neoplasms, NOS
    8046: ("Epithelial neoplasms, NOS", "Carcinoma"),
    # Papillary Carcinoma, NOS
    8050: ("Papillary carcinoma in situ", "Carcinoma"),
    8051: ("Verrucous carcinoma, NOS", "Carcinoma"),
    8052: ("Papillary squamous cell carcinoma, non-invasive", "Carcinoma"),
    8053: ("Papillary squamous cell carcinoma", "Carcinoma"),
    # Squamous Cell Carcinoma, NOS
    8070: ("Squamous cell carcinoma, NOS", "Carcinoma"),
    8071: ("Sq. cell carcinoma, keratinizing, NOS", "Carcinoma"),
    8072: ("Sq. cell carcinoma, large cell, non-ker.", "Carcinoma"),
    8073: ("Sq. cell carcinoma, small cell, non-ker.", "Carcinoma"),
    8074: ("Sq. cell carcinoma, spindle cell", "Carcinoma"),
    8075: ("Squamous cell carcinoma, clear cell", "Carcinoma"),
    8076: ("Sq. cell carcinoma in situ with question. stromal invas.", "Carcinoma"),
    8077: ("Sq. cell carcinoma, micro-invasive", "Carcinoma"),
    8078: ("Squamous cell carcinoma with horn formation", "Carcinoma"),
    8082: ("Lymphoepithelial carcinoma", "Carcinoma"),
    8110: ("Pilomatrix carcinoma", "Carcinoma"),
    8123: ("Basaloid carcinoma", "Carcinoma"),
    8130: ("Papillary transitional cell carcinoma","Carcinoma"),
    # Adenocarcinoma, NOS
    8140: ("Adenocarcinoma in situ", "Adenocarcinoma"),
    8141: ("Adenocarcinoma, NOS", "Adenocarcinoma"),
    8143: ("Superficial spreading adenocarcinoma", "Adenocarcinoma"),
    8147: ("Basal cell adenocarcinoma", "Adenocarcinoma"),
    # Trabecular Adenocarcinoma
    8190: ("Trabecular adenocarcinoma", "Adenocarcinoma"),
    # Adenoid Cystic & Cribriform Ca.
    8200: ("Adenoid cystic carcinoma", "Adenocarcinoma"),
    8201: ("Cribriform carcinoma in situ", "Adenocarcinoma"),
    8202: ("Cribriform carcinoma", "Adenocarcinoma"),
    # Adenoca. in Adenoma Polyp
    8211: ("Tubular adenocarcinoma", "Adenocarcinoma"),
    # Solid Carcinoma, NOS
    8230: ("Duct carcinoma in situ, solid type", "Carcinoma"),
    8231: ("Carcinoma simplex", "Carcinoma"),
    # Carcinoid Tumor, Malignant
    8240: ("Carcinoid tumor, malignant", "Carcinoid"),
    8243: ("Goblet cell carcinoid", "Carcinoid"),
    8246: ("Neuroendocrine carcinoma", "Carcinoid"),
    8249: ("Neuroendocrine tumor", "Carcinoid"),
    # Bronchiolo-Alveolar Adenoca.
    8251: ("Alveolar adenocarcinoma", "Adenocarcinoma"),
    8255: ("Adenocarcinoma with mixed subtypes", "Adenocarcinoma"),
    # Papillary Adenocarcinoma, NOS
    8260: ("Papillary adenocarcinoma, NOS", "Adenocarcinoma"),
    8261: ("Adenocarcinoma in situ in villous adenoma", "Adenocarcinoma"),
    8263: ("Adenocarcinoma in villous adenoma", "Adenocarcinoma"),
    # Oxyphilic Adenocarcinoma
    8290: ("Oxyphilic adenocarcinoma", "Adenocarcinoma"),
    # Clear Cell Adenocarcinoma, NOS
    8310: ("Clear cell adenocarcinoma, NOS", "Adenocarcinoma"),
    8314: ("Lipid-rich carcinoma", "Adenocarcinoma"),
    8315: ("Glycogen-rich carcinoma", "Adenocarcinoma"),
    # Granular Cell Carcinoma
    8320: ("Granular cell carcinoma", "Carcinoma"),
    8323: ("Mixed cell adenocarcinoma", "Adenocarcinoma"),
    # Papillary Carcinoma
    8341: ("Papillary microcarcinoma", "Carcinoma"),
    8343: ("Papillary carcinoma, encapsulated", "Carcinoma"),
    8344: ("Papillary carcinoma, columnar cell", "Carcinoma"),
    # Sweat Gland Adenocarcinoma
    8401: ("Apocrine adenocarcinoma", "Adenocarcinoma"),
    # Sebaceous/Eccrine Adenoca.
    8410: ("Sebaceous adenocarcinoma", "Adenocarcinoma"),
    # Mucoepidermoid Carcinoma
    8430: ("Mucoepidermoid carcinoma", "Carcinoma"),
    # Cystadenocarcinoma, NOS
    8440: ("Cystadenocarcinoma, NOS", "Adenocarcinoma"),
    # Intraductal papillary‑mucinous carcinoma
    8453: ("Intraductal papillary‑mucinous carcinoma", "Carcinoma"),
    8460: ("Papillary serous cystadenocarcinoma, NOS", "Carcinoma"),
    # Mucinous Cystadenocarcinoma, NOS
    8470: ("Mucinous cystadenocarcinoma, NOS", "Adenocarcinoma"),
    8471: ("Papillary mucinous cystadenocarcinoma","Adenocarcinoma"),
    # Mucinous Adenocarcinoma
    8480: ("Mucinous adenocarcinoma", "Adenocarcinoma"),
    8481: ("Mucin-producing adenocarcinoma", "Adenocarcinoma"),
    # Signet Ring Cell Carcinoma
    8490: ("Signet ring cell carcinoma", "Carcinoma"),
    # Duct Carcinoma
    8500: ("Intraductal carcinoma, noninfiltrating, NOS", "Carcinoma"),
    8501: ("Invasive carcinoma of no special type", "Carcinoma"),
    8502: ("Comedocarcinoma, non-infiltrating", "Carcinoma"),
    8503: ("Comedocarcinoma, NOS", "Carcinoma"),
    8504: ("Intracystic carcinoma, NOS", "Carcinoma"),
    8507: ("Invasive micropapillary carcinoma", "Carcinoma"),
    8508: ("Cystic hypersecretory carcinoma", "Carcinoma"),
    8509: ("Solid papillary carcinoma in situ", "Carcinoma"),
    # Medullary Carcinoma, NOS
    8510: ("Medullary carcinoma, NOS", "Carcinoma"),
    8512: ("Medullary carcinoma with lymphoid stroma", "Carcinoma"),
    8513: ("Atypical medullary carcinoma", "Carcinoma"),
    8514: ("Duct carcinoma, desmoplastic type", "Carcinoma"),
    8519: ("Pleomorphic lobular carcinoma in situ", "Carcinoma"),
    # Lobular and Other Ductal Ca.
    8520: ("Lobular carcinoma in situ", "Carcinoma"),
    8521: ("Infiltrating ductular carcinoma", "Carcinoma"),
    8522: ("Intraductal and lobular in situ carcinoma", "Carcinoma"),
    8523: ("Infiltrating duct mixed with other types of carcinoma, in situ", "Carcinoma"),
    8524: ("Infiltrating lobular mixed with other types of carcinoma", "Carcinoma"),
    8525: ("Polymorphous low grade adenocarcinoma", "Adenocarcinoma"),
    # Inflammatory Carcinoma
    8530: ("Inflammatory carcinoma", "Carcinoma"),
    # Paget Disease, Mammary
    8540: ("Paget disease, mammary", "Paget disease"),
    8541: ("Paget disease and ductal carcinoma", "Paget disease"),
    8542: ("Paget disease, extramammary", "Paget disease"),
    8543: ("Paget disease and intraductal carcinoma", "Paget disease"),
    # Acinar Cell Carcinoma
    8550: ("Acinar cell carcinoma", "Carcinoma"),
    8551: ("Acinar cell cystadenocarcinoma", "Carcinoma"),
    # Adenosquamous Carcinoma
    8560: ("Adenosquamous carcinoma", "Carcinoma"),
    8562: ("Epithelial-myoepithelial carcinoma", "Carcinoma"),
    # Adenoca. with Metaplasia
    8570: ("Adenocarcinoma with squamous metaplasia", "Adenocarcinoma"),
    8571: ("Adenocarcinoma with cartilage & oss. metaplasia", "Adenocarcinoma"),
    8572: ("Adenocarcinoma with spindle cell metaplasia", "Adenocarcinoma"),
    8573: ("Adenocarcinoma with apocrine metaplasia", "Adenocarcinoma"),
    8574: ("Adenocarcinoma with neuroendocrine differentiation", "Adenocarcinoma"),
    8575: ("Metaplastic carcinoma, NOS", "Carcinoma"),
    # Malignant Melanoma, NOS
    8720: ("Malignant melanoma, NOS", "Melanoma"),
    # Sarcoma, NOS
    8800: ("Sarcoma, NOS", "Sarcoma"),
    8801: ("Spindle cell sarcoma", "Sarcoma"),
    8802: ("Giant cell sarcoma", "Sarcoma"),
    8803: ("Small cell sarcoma", "Sarcoma"),
    8804: ("Epitheloid sarcoma", "Sarcoma"),
    8805: ("Undifferentiated sarcoma", "Sarcoma"),
    8806: ("Desmoplastic small round cell tumor", "Sarcoma"),
    # Fibromatous Neoplasms
    8810: ("Fibrosarcoma, NOS", "Sarcoma"),
    8811: ("Fibromyxosarcoma", "Sarcoma"),
    8813: ("Infantile fibrosarcoma", "Sarcoma"),
    8815: ("Solitary fibrous tumor, malignant", "Sarcoma"),
    # Sarcoma, NOS
    8825: ("Myofibroblastic sarcoma", "Sarcoma"),
    # Fibrous Histiocytoma
    8830: ("Fibrous histiocytoma", "Sarcoma"),
    8832: ("Dermatofibrosarcoma", "Sarcoma"),
    # Liposarcoma Neoplasms
    8850: ("Liposarcoma, NOS", "Sarcoma"),
    8851: ("Liposarcoma, well differentiated", "Sarcoma"),
    8852: ("Myxoid liposarcoma", "Sarcoma"),
    8853: ("Round cell liposarcoma", "Sarcoma"),
    8854: ("Pleomorphic liposarcoma", "Sarcoma"),
    8855: ("Mixed type liposarcoma", "Sarcoma"),
    8857: ("Fibroblastic liposarcoma", "Sarcoma"),
    8858: ("Dedifferentiated liposarcoma", "Sarcoma"),
    # Myomatous Neoplasms
    8890: ("Leiomyosarcoma, NOS", "Sarcoma"),
    8891: ("Epitheloid leiomyosarcoma", "Sarcoma"),
    8894: ("Angiomyosarcoma", "Sarcoma"),
    8896: ("Myxoid leiomyosarcoma", "Sarcoma"),
    8900: ("Rhabdomyosarcoma, NOS", "Sarcoma"),
    8920: ("Alveolar rhabdomyosarcoma", "Sarcoma"),
    # Stromal Sarcoma
    8935: ("Stromal sarcoma", "Sarcoma"),
    8940: ("Mixed tumor, malignant, NOS", "Carcinoma"),
    # Carcinosarcoma, NOS
    8980: ("Carcinosarcoma, NOS", "Sarcoma"),
    8981: ("Carcinosarcoma, embryonal type", "Sarcoma"),
    8982: ("Malignant myoepithelioma", "Sarcoma"),
    8983: ("Adenomyoepithelioma with carcinoma", "Sarcoma"),
    # Mesenchymoma, Malignant
    8990: ("Mesenchymoma, malignant", "Sarcoma"),
    8991: ("Embryonal sarcoma", "Sarcoma"),
    # Phyllodes Tumor, Malignant
    9020: ("Phyllodes tumor, malignant", "Sarcoma"),
    # Synovial Sarcoma, NOS
    9040: ("Synovial sarcoma, NOS", "Sarcoma"),
    9041: ("Synovial sarcoma, spindle cell", "Sarcoma"),
    # Blood Vessel Tumors
    9120: ("Hemangiosarcoma", "Sarcoma"),
    # Hemangioendothelioma
    9130: ("Hemangioendothelioma, malignant", "Sarcoma"),
    9133: ("Epitheloid hemangioendothelioma, malignant", "Sarcoma"),
    # Osteosarcoma, NOS	
    9170: ("Lymphangiosarcoma", "Sarcoma"),
    9180: ("Osteosarcoma, NOS", "Sarcoma"),
    9183: ("Telangiectatic osteosarcoma", "Sarcoma"),
    9220: ("Chondrosarcoma, NOS", "Sarcoma"),
    # Granular Cell Tumor
    9580: ("Granular cell tumor, malignant", "Sarcoma"),
    9581: ("Alveolar soft part sarcoma", "Sarcoma"),
    # Malignant Lymphoma, NOS
    9590: ("Malignant lymphoma, NOS", "Lymphoma"),
    9591: ("Malignant lymphoma, non-Hodgkin", "Lymphoma"),
    9596: ("Composite Hodgkin and non-Hodgkin lymphoma", "Lymphoma"),
    # Hodgkin Lymphoma
    9650: ("Hodgkin lymphoma, lymphocyte-rich", "Lymphoma"),
    9651: ("Hodgkin tumor, mixed cellularity, NOS", "Lymphoma"),
    9652: ("Hodgkin lymphoma, lymphocyte depleted, NOS", "Lymphoma"),
    9653: ("Hodgkin lymphoma, lymphocyte depleted, diffuse fibrosis", "Lymphoma"),
    9654: ("Hodgkin lymphoma, lymphocyte predominant", "Lymphoma"),
    9655: ("Hodgkin lymphoma, nodular lymphocyte predom.", "Lymphoma"),
    # Hodgkin Lymphoma, Nod. Scler.
    9661: ("Hodgkin sarcoma [obs]", "Lymphoma"),
    9663: ("Hodgkin lymphoma, nodular sclerosis, NOS", "Lymphoma"),
    9664: ("Hodgkin lymphoma, nod. scler., cellular phase", "Lymphoma"),
    9665: ("Hodgkin lymphoma, nod. scler., grade 1", "Lymphoma"),
    9667: ("Hodgkin lymphoma, nodular scler., grade 2", "Lymphoma"),
    # ML, Small B-cell Lymphocytic
    9670: ("ML, small B lymphocytic, NOS", "Lymphoma"),
    9671: ("ML, lymphoplasmacytic", "Lymphoma"),
    9673: ("Mantle cell lymphoma", "Lymphoma"),
    9675: ("ML, mixed sm. and lg. cell, diffuse", "Lymphoma"),
    # ML, Large B-cell, Diffuse
    9680: ("ML, large B-cell, diffuse", "Lymphoma"),
    9684: ("ML, large B-cell, diffuse, immunoblastic", "Lymphoma"),
    9687: ("Burkitt lymphoma, NOS", "Lymphoma"),
    9688: ("T-cell histiocyte rich large B-cell lymphoma", "Lymphoma"),
    # Follic. & Marginal Lymph, NOS
    9690: ("Follicular lymphoma, NOS", "Lymphoma"),
    9691: ("Follicular lymphoma, grade 2", "Lymphoma"),
    9695: ("Follicular lymphoma, grade 1", "Lymphoma"),
    9698: ("Follicular lymphoma, grade 3", "Lymphoma"),
    9699: ("Marginal zone B-cell lymphoma, NOS", "Lymphoma"),
    # T-cell Lymphomas
    9701: ("Sezary syndrome", "Lymphoma"),
    9705: ("Angioimmunoblastic T-cell lymphoma", "Lymphoma"),
    # Other Spec. Non-Hodgkin Lymphoma
    9712: ("Intravascular large B-cell lymphoma", "Lymphoma"),
    9714: ("Anaplastic large cell lymphoma, T-cell and Null cell type", "Lymphoma"),
    9716: ("Anaplastic large cell lymphoma, ALK negative", "Lymphoma"),
    9719: ("NK/T-cell lymphoma, nasal and nasal-type", "Lymphoma"),
    # Precurs. Cell Lymphoblastic Lymph.
    9724: ("Systemic EBV pos. T-cell lymphoproliferative disease of childhood", "Lymphoma"),
    9727: ("Precursor cell lymphoblastic lymphoma, NOS", "Lymphoma"),
    9728: ("Precursor B-cell lymphoblastic lymphoma", "Lymphoma"),
    9729: ("Precursor T-cell lymphoblastic lymphoma", "Lymphoma"),
    # Plasma Cell Tumors
    9731: ("Plasmacytoma, NOS", "Lymphoma"),
    9733: ("Plasmablastic lymphoma", "Lymphoma"),
    9734: ("ALK positive large B-cell lymphoma", "Lymphoma"),
    9735: ("Lrg B-cell lymphoma in HHV8-assoc. multicentric Castleman DZ", "Lymphoma"),
    9738: ("Plasma cell myeloma", "Lymphoma"),
    # Mast Cell Tumors
    9740: ("Mast cell sarcoma", "Lymphoma"),
    9741: ("Malignant mastocytosis", "Lymphoma"),
    9743: ("Erdheim-Chester Disease", "Lymphoma"),
    # Neoplasms of Histiocytes & Accessory Lymphoid Cells
    9750: ("Malignant histiocytosis", "Lymphoma"),
    9751: ("Langerhans cell histiocytosis, NOS", "Lymphoma"),
    9754: ("Langerhans cell histiocytosis, disseminated", "Lymphoma"),
    9755: ("Histiocytic sarcoma", "Lymphoma"),
    9756: ("Langerhans cell sarcoma", "Lymphoma"),
    9757: ("Interdigitating dendritic cell sarcoma", "Lymphoma"),
    9758: ("Follicular dendritic cell sarcoma", "Lymphoma"),
    9759: ("Fibroblastic reticular cell tumor", "Lymphoma"),
    # Immunoproliferative Diseases
    9766: ("Immunoproliferative diseases, grade 3", "Lymphoma"),
    # Precursor Lymphoid Neoplasms
    9811: ("B lymphoblastic leukemia/lymphoma, NOS", "Leukemia/Lymphoma"),
    9813: ("Leukemia/lymphoma with t(9;22)(q34;q11.2);BCR-ABL1", "Leukemia/Lymphoma"),
    9814: ("Leukemia/lymphoma with t(12;21)(p13;q22);TEL-AML1(ETV6-RUNX1)", "Leukemia/Lymphoma"),
    9815: ("B lymphoblastic leukemia/lymphoma with hyperdiploidy", "Leukemia/Lymphoma"),
    9816: ("Leukemia/lymphoma with hypodiploidy (hypodiploid ALL)", "Leukemia/Lymphoma"),
    # Lymphoid Leukemia, NOS
    9823: ("Chronic lymphocytic leukemia/small lymphocytic lymphoma", "Leukemia/Lymphoma"),
    # Prolymph/Precurs Leukemia
    9831: ("T-cell large granular lymphocytic leukemia", "Leukemia/Lymphoma"),
    9837: ("T lymphoblastic leukemia/lymphoma", "Leukemia/Lymphoma"),
    # Chronic Myeloproliferative Dis.
    9965: ("Myeloid and lymphoid neoplasms with PDGFRB rearrangement", "Leukemia/Lymphoma"),
    9967: ("Myeloid and lymphoid neoplasm with FGFR1 abnormalities", "Leukemia/Lymphoma"),
    # Myeloplastic/Myeloproliferative Neoplasms
    9971: ("Polymorphic PTLD", "Leukemia/Lymphoma"),
    9975: ("Myelodysplastic/Myeloproliferative neoplasm, unclassifiable", "Leukemia/Lymphoma"),
}

In [26]:
def get_histology_name(code):
    return histology_mapping[code][0]

def get_histology_category(code):
    return histology_mapping[code][1]

In [27]:
df['Histology_Name'] = df['Histologic_Type'].apply(get_histology_name)
df['Histology_Category'] = df['Histologic_Type'].apply(get_histology_category)

In [28]:
df.head(20)

,Patient_ID,YearDx,AgeDx,Race,Origin,Histologic_Type,Stage,Grade,Survival_months,Vital_status_recode,Histology_Name,Histology_Category
0,321,2015,54,White,Non-Spanish-Hispanic-Latino,8522,Regional lymph nodes involved only,Moderately differentiated; Grade II,12.0,Alive,Intraductal and lobular in situ carcinoma,Carcinoma
1,357,2013,44,White,Non-Spanish-Hispanic-Latino,8010,Unknown/unstaged/unspecified/DCO,Unknown,28.0,Dead,"Carcinoma, NOS",Carcinoma
2,390,2012,82,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,51.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma
3,405,2014,56,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,33.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma
4,581,2013,71,White,Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,38.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma
5,766,2015,75,White,Non-Spanish-Hispanic-Latino,8500,Distant site(s)/node(s) involved,Unknown,17.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma
6,945,2015,87,White,Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,7.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma
7,1080,2016,70,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Well differentiated; Grade I,3.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma
8,2008,2015,84,White,Non-Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,16.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma
9,2532,2015,61,Black,Non-Spanish-Hispanic-Latino,8510,Localized only,Poorly differentiated; Grade III,22.0,Alive,"Medullary carcinoma, NOS",Carcinoma


In [29]:
def categorized_race(row):
    if row['Race'] == 'White':
        if row['Origin'] == 'Spanish-Hispanic-Latino':
            return 'Hispanic White'
        else:
            return 'Non-Hispanic White'
    elif row['Race'] == 'Black':
        return 'Black'
    elif row['Race'] == 'Asian or Pacific Islander':
        return 'Asian or Pacific Islander'
    else:
        return 'Other'

df['Race_Group'] = df.apply(categorized_race, axis=1)
df = df[df['Race_Group'].isin(['Hispanic White','Non-Hispanic White','Black','Asian or Pacific Islander'])]
df.head(20)

,Patient_ID,YearDx,AgeDx,Race,Origin,Histologic_Type,Stage,Grade,Survival_months,Vital_status_recode,Histology_Name,Histology_Category,Race_Group
0,321,2015,54,White,Non-Spanish-Hispanic-Latino,8522,Regional lymph nodes involved only,Moderately differentiated; Grade II,12.0,Alive,Intraductal and lobular in situ carcinoma,Carcinoma,Non-Hispanic White
1,357,2013,44,White,Non-Spanish-Hispanic-Latino,8010,Unknown/unstaged/unspecified/DCO,Unknown,28.0,Dead,"Carcinoma, NOS",Carcinoma,Non-Hispanic White
2,390,2012,82,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,51.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White
3,405,2014,56,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,33.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White
4,581,2013,71,White,Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,38.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Hispanic White
5,766,2015,75,White,Non-Spanish-Hispanic-Latino,8500,Distant site(s)/node(s) involved,Unknown,17.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White
6,945,2015,87,White,Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,7.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Hispanic White
7,1080,2016,70,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Well differentiated; Grade I,3.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White
8,2008,2015,84,White,Non-Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,16.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White
9,2532,2015,61,Black,Non-Spanish-Hispanic-Latino,8510,Localized only,Poorly differentiated; Grade III,22.0,Alive,"Medullary carcinoma, NOS",Carcinoma,Black


In [31]:
def categorized_grade(row):
    if row['Grade'] == 'Well differentiated; Grade I':
        return 'Grade I: Well differentiated'
    elif row['Grade'] == 'Moderately differentiated; Grade II':
        return 'Grade II: Moderately differentiated'
    elif row['Grade'] == 'Poorly differentiated; Grade III':
        return 'Grade III: Poorly differentiated'
    elif row['Grade'] == 'Undifferentiated; anaplastic; Grade IV':
        return 'Grade IV: Undifferentiated; anaplastic'
    else:
        return 'Unknown'

df['Grade_Type'] = df.apply(categorized_grade, axis=1)
df.head(10)

,Patient_ID,YearDx,AgeDx,Race,Origin,Histologic_Type,Stage,Grade,Survival_months,Vital_status_recode,Histology_Name,Histology_Category,Race_Group,Grade_Type
0,321,2015,54,White,Non-Spanish-Hispanic-Latino,8522,Regional lymph nodes involved only,Moderately differentiated; Grade II,12.0,Alive,Intraductal and lobular in situ carcinoma,Carcinoma,Non-Hispanic White,Grade II: Moderately differentiated
1,357,2013,44,White,Non-Spanish-Hispanic-Latino,8010,Unknown/unstaged/unspecified/DCO,Unknown,28.0,Dead,"Carcinoma, NOS",Carcinoma,Non-Hispanic White,Unknown
2,390,2012,82,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,51.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade II: Moderately differentiated
3,405,2014,56,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,33.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade II: Moderately differentiated
4,581,2013,71,White,Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,38.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Hispanic White,Grade II: Moderately differentiated
5,766,2015,75,White,Non-Spanish-Hispanic-Latino,8500,Distant site(s)/node(s) involved,Unknown,17.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Unknown
6,945,2015,87,White,Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,7.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Hispanic White,Grade III: Poorly differentiated
7,1080,2016,70,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Well differentiated; Grade I,3.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade I: Well differentiated
8,2008,2015,84,White,Non-Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,16.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade III: Poorly differentiated
9,2532,2015,61,Black,Non-Spanish-Hispanic-Latino,8510,Localized only,Poorly differentiated; Grade III,22.0,Alive,"Medullary carcinoma, NOS",Carcinoma,Black,Grade III: Poorly differentiated


In [32]:
def categorized_age(row):
    if row['AgeDx'] < 40:
        return '<40'
    elif row['AgeDx'] >= 40 and row['AgeDx'] < 50:
        return '40-49'
    elif row['AgeDx'] >= 50 and row['AgeDx'] < 60:
        return '50-59'
    elif row['AgeDx'] >= 60 and row['AgeDx'] < 70:
        return '60-69'
    elif row['AgeDx'] >= 70 and row['AgeDx'] < 80:
        return '70-79'
    else:
        return '80+'

df['Age_Group'] = df.apply(categorized_age, axis=1)
df.head(10)

,Patient_ID,YearDx,AgeDx,Race,Origin,Histologic_Type,Stage,Grade,Survival_months,Vital_status_recode,Histology_Name,Histology_Category,Race_Group,Grade_Type,Age_Group
0,321,2015,54,White,Non-Spanish-Hispanic-Latino,8522,Regional lymph nodes involved only,Moderately differentiated; Grade II,12.0,Alive,Intraductal and lobular in situ carcinoma,Carcinoma,Non-Hispanic White,Grade II: Moderately differentiated,50-59
1,357,2013,44,White,Non-Spanish-Hispanic-Latino,8010,Unknown/unstaged/unspecified/DCO,Unknown,28.0,Dead,"Carcinoma, NOS",Carcinoma,Non-Hispanic White,Unknown,40-49
2,390,2012,82,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,51.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade II: Moderately differentiated,80+
3,405,2014,56,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,33.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade II: Moderately differentiated,50-59
4,581,2013,71,White,Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,38.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Hispanic White,Grade II: Moderately differentiated,70-79
5,766,2015,75,White,Non-Spanish-Hispanic-Latino,8500,Distant site(s)/node(s) involved,Unknown,17.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Unknown,70-79
6,945,2015,87,White,Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,7.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Hispanic White,Grade III: Poorly differentiated,80+
7,1080,2016,70,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Well differentiated; Grade I,3.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade I: Well differentiated,70-79
8,2008,2015,84,White,Non-Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,16.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade III: Poorly differentiated,80+
9,2532,2015,61,Black,Non-Spanish-Hispanic-Latino,8510,Localized only,Poorly differentiated; Grade III,22.0,Alive,"Medullary carcinoma, NOS",Carcinoma,Black,Grade III: Poorly differentiated,60-69


In [33]:
def categorized_status(row):
    if row['Vital_status_recode'] == 'Alive':
        return 0
    else:
        return 1

df['Status'] = df.apply(categorized_status, axis=1)
df.head(10)

,Patient_ID,YearDx,AgeDx,Race,Origin,Histologic_Type,Stage,Grade,Survival_months,Vital_status_recode,Histology_Name,Histology_Category,Race_Group,Grade_Type,Age_Group,Status
0,321,2015,54,White,Non-Spanish-Hispanic-Latino,8522,Regional lymph nodes involved only,Moderately differentiated; Grade II,12.0,Alive,Intraductal and lobular in situ carcinoma,Carcinoma,Non-Hispanic White,Grade II: Moderately differentiated,50-59,0
1,357,2013,44,White,Non-Spanish-Hispanic-Latino,8010,Unknown/unstaged/unspecified/DCO,Unknown,28.0,Dead,"Carcinoma, NOS",Carcinoma,Non-Hispanic White,Unknown,40-49,1
2,390,2012,82,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,51.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade II: Moderately differentiated,80+,0
3,405,2014,56,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,33.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade II: Moderately differentiated,50-59,0
4,581,2013,71,White,Spanish-Hispanic-Latino,8500,Localized only,Moderately differentiated; Grade II,38.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Hispanic White,Grade II: Moderately differentiated,70-79,0
5,766,2015,75,White,Non-Spanish-Hispanic-Latino,8500,Distant site(s)/node(s) involved,Unknown,17.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Unknown,70-79,1
6,945,2015,87,White,Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,7.0,Dead,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Hispanic White,Grade III: Poorly differentiated,80+,1
7,1080,2016,70,White,Non-Spanish-Hispanic-Latino,8500,Localized only,Well differentiated; Grade I,3.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade I: Well differentiated,70-79,0
8,2008,2015,84,White,Non-Spanish-Hispanic-Latino,8500,Regional lymph nodes involved only,Poorly differentiated; Grade III,16.0,Alive,"Intraductal carcinoma, noninfiltrating, NOS",Carcinoma,Non-Hispanic White,Grade III: Poorly differentiated,80+,0
9,2532,2015,61,Black,Non-Spanish-Hispanic-Latino,8510,Localized only,Poorly differentiated; Grade III,22.0,Alive,"Medullary carcinoma, NOS",Carcinoma,Black,Grade III: Poorly differentiated,60-69,0


In [34]:
df.to_csv("cleaned_SEER.csv", index=False)